In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Read the data
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
holidays_events = pd.read_csv("data/holidays_events.csv")
oil = pd.read_csv("data/oil.csv")

stores = pd.read_csv("data/stores.csv")
transactions = pd.read_csv("data/transactions.csv")

# Pre-processing

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [6]:
# Pre-processing
# Convert date columns to datetime format
train["date"] = pd.to_datetime(train["date"])
test["date"] = pd.to_datetime(test["date"])
holidays_events["date"] = pd.to_datetime(holidays_events["date"])
oil["date"] = pd.to_datetime(oil["date"])
transactions["date"] = pd.to_datetime(transactions["date"])

train = train.merge(oil, on="date", how="left")
train = train.merge(transactions, on=["date", "store_nbr"], how="left")
test = test.merge(oil, on="date", how="left")
test = test.merge(transactions, on=["date", "store_nbr"], how="left")

# Feature engineering
# Create a simple feature for holidays
train["is_holiday"] = train["date"].isin(holidays_events["date"]).astype(int)
test["is_holiday"] = test["date"].isin(holidays_events["date"]).astype(int)

# Fill missing values
train["dcoilwtico"].fillna(method="ffill", inplace=True)
test["dcoilwtico"].fillna(method="ffill", inplace=True)
train["transactions"].fillna(train["transactions"].mean(), inplace=True)
test["transactions"].fillna(train["transactions"].mean(), inplace=True)

/var/folders/g6/ld3hdqzj3d7840n3yw1hgcjc0000gn/T/ipykernel_63969/3810366704.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train["dcoilwtico"].fillna(method="ffill", inplace=True)
/var/folders/g6/ld3hdqzj3d7840n3yw1hgcjc0000gn/T/ipykernel_63969/3810366704.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test["dcoilwtico"].fillna(method="ffill", inplace=True)


In [26]:
example_train = train[(train["store_nbr"] == 1) & (train["family"] == "AUTOMOTIVE")]
example_test = test[(test["store_nbr"] == 1) & (test["family"] == "AUTOMOTIVE")]
example_train

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico_x,transactions_x,dcoilwtico_y,transactions_y,is_holiday,dcoilwtico,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,NaN,NaN,NaN,1,NaN,1694.602158
1782,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,93.14,2111.0,93.14,2111.0,0,93.14,2111.000000
3564,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,92.97,1833.0,92.97,1833.0,0,92.97,1833.000000
5346,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,93.12,1863.0,93.12,1863.0,0,93.12,1863.000000
7128,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,NaN,1509.0,NaN,1509.0,1,93.12,1509.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991978,2991978,2017-08-11,1,AUTOMOTIVE,1.0,0,48.81,570.0,48.81,570.0,1,48.81,570.000000
2993760,2993760,2017-08-12,1,AUTOMOTIVE,6.0,0,NaN,1004.0,NaN,1004.0,0,48.81,1004.000000
2995542,2995542,2017-08-13,1,AUTOMOTIVE,1.0,0,NaN,416.0,NaN,416.0,0,48.81,416.000000
2997324,2997324,2017-08-14,1,AUTOMOTIVE,1.0,0,47.59,1733.0,47.59,1733.0,0,47.59,1733.000000


In [35]:
model = ARIMA(example_train["sales"], order=(1, 1, 1))
fitted_model = model.fit()

# Predict the sales for the test dataset
predictions = fitted_model.forecast(steps=len(example_test))

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/hxl/miniconda/envs/store-sales/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/hxl/miniconda/envs/store-sales/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/hxl/miniconda/envs/store-sales/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/hxl/miniconda/envs/store-sales/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/hxl/miniconda/envs/store-sales/lib/python3.11

1684    4.453321
1685    4.454191
1686    4.454192
1687    4.454193
1688    4.454193
1689    4.454193
1690    4.454193
1691    4.454193
1692    4.454193
1693    4.454193
1694    4.454193
1695    4.454193
1696    4.454193
1697    4.454193
1698    4.454193
1699    4.454193
Name: predicted_mean, dtype: float64

In [34]:
submission = test[["id"]].copy()
submission["sales"] = 0  # Initialize all sales to 0
# Update the predictions for the example product family
submission.loc[example_test.index, "sales"] = predictions
submission = submission[submission["sales"] != 0]
submission
predictions

# submission.to_csv("submission.csv", index=False)
# predictions

1684    4.453321
1685    4.454191
1686    4.454192
1687    4.454193
1688    4.454193
1689    4.454193
1690    4.454193
1691    4.454193
1692    4.454193
1693    4.454193
1694    4.454193
1695    4.454193
1696    4.454193
1697    4.454193
1698    4.454193
1699    4.454193
Name: predicted_mean, dtype: float64